In [35]:
import os
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List
import pandas as pd

In [43]:
model_path = '/home/varun/Desktop/KeyPointDetection/runs/pose/train16/weights/best.pt'
img_folder = '/home/varun/Desktop/KeyPointDetection/runs/pose/train16/Frames_Images/data_one/data'  # Update this to the path of your image folder
output_excel_path = 'angles_results1.xlsx'


In [45]:
all_results_df = pd.DataFrame(columns=['File', 'Angle_ABC', 'Angle_BDC'])

In [46]:
def compute_angle(A, B, C):
    # Vectors for BA and BC
    a = [A[0] - B[0], A[1] - B[1]]
    b = [C[0] - B[0], C[1] - B[1]]
    c = [A[0] - C[0], A[1] - C[1]]

    # Magnitudes (lengths) of BA, BC, and AC
    ma = math.sqrt(a[0] ** 2 + a[1] ** 2)
    mb = math.sqrt(b[0] ** 2 + b[1] ** 2)
    mc = math.sqrt(c[0] ** 2 + c[1] ** 2)

    print(ma)
    print(mb)
    print(mc)
    # Using the law of cosines to find the angle at B
    cosB = (ma ** 2 + mc ** 2 - mb ** 2) / (2 * ma * mc)

    # Angle in radians
    angle = math.acos(cosB)

    # Convert angle to degrees
    angle_deg = math.degrees(angle)

    return angle_deg

In [47]:
def calculate_pointer_angle(img_path):
    # Calculates the Pointer Angle
    # separate coordinates from box

    # Detect the gauge and get the pointer angle
    model1 = YOLO(model_path)
    results = model1(img_path)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()
        print(keypoints_set)

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        print(keypoints_set[0])
        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]
        print(keypoints_set[0])
        print(f"A is {A[1]}")
        print(f"B is {B[1]}")
        print(f"C is {C[1]}")

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")
    return theta1, theta2

In [48]:
def save_to_excel(file_name, angle_ABC, angle_BDC):
    global all_results_df
    data = {'File': [file_name], 'Angle_ABC': [angle_ABC], 'Angle_BDC': [angle_BDC]}
    df = pd.DataFrame(data)
    
    # Append to the global DataFrame
    all_results_df = all_results_df.append(df, ignore_index=True)

    # Save to Excel
    all_results_df.to_excel(output_excel_path, index=False)

In [49]:
if __name__ == "__main__":
    for file_name in os.listdir(img_folder):
        if file_name.endswith(('.jpg', '.png', '.jpeg')):  # Add other image formats if needed
            img_path = os.path.join(img_folder, file_name)
            image_output = calculate_pointer_angle(img_path)
            save_to_excel(file_name, *image_output)


image 1/1 /home/varun/Desktop/KeyPointDetection/runs/pose/train16/Frames_Images/data_one/data/frame7116.jpg: 640x640 2 KEYPOINT_SKELETONs, 26.1ms
Speed: 3.0ms preprocess, 26.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
[[[     245.77      252.21]
  [     147.96      357.58]
  [     355.31      339.78]
  [     121.88      326.37]]

 [[      187.1      270.81]
  [     115.55      341.76]
  [     254.31      338.79]
  [     77.499       288.3]]]
[[     245.77      252.21]
 [     147.96      357.58]
 [     355.31      339.78]
 [     121.88      326.37]]
[[     245.77      252.21]
 [     147.96      357.58]
 [     355.31      339.78]
 [     121.88      326.37]]
A is 252.21292114257812
B is 357.576904296875
C is 326.3743591308594
143.76586611126584
40.66807925221532
144.3938527454585
208.1092414036149
233.81412156761678
40.66807925221532
Angle between keypoints A, B, and C is: 16.22 degrees
Angle between keypoints B, C, and D is: 235.01 degrees


AttributeError: 'DataFrame' object has no attribute 'append'